In [1]:
!pip install requests_html
import pandas as pd
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from time import sleep
from random import randint

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.4 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=f2b92cd27f7eb55bc6c0ee42b5b69da2d99d246cb8b2beaa53c28a5cdbb33728
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [4]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd

s = HTMLSession()
url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1'
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

def getdata(url):
    r = s.get(url, headers=headers)  # Adjust if you're not using proxies
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n" % url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d" % (url, r.status_code))
        return None
    soup = BeautifulSoup(r.html.html, 'html.parser')
    return soup

def getNextPage(soup):
    page = soup.find('span', {'class': 's-pagination-strip'})
    if not page.find('a', {'class': 's-pagination-item s-pagination-next s-pagination-disabled '}):
        link = page.find('a', {'class': 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'})
        if not link:
            return
        url = 'http://www.amazon.in' + str(link['href'])
        return url
    else:
        return

pageUrls = []
while True:
    pageUrls.append(url)
    soup = getdata(url)
    url = getNextPage(soup)
    if not url:
        break

data = []

def appendInfo(content):
    for item in content:
        link = item.find('a', {'class': 'a-link-normal s-no-outline'})
        url = 'https://www.amazon.in' + str(link['href'])
        name = item.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
        price = item.find('span', {'class': 'a-offscreen'})
        rating = item.find('span', {'class': 'a-icon-alt'})
        review = item.find('span', {'class': 'a-size-base s-underline-text'})
        if not rating:
            rating = "-"
            review = "-"
        else:
            rating = rating.text
            review = review.text
            review = str(review)
            review = review.replace("(", "")
            review = review.replace(")", "")
        data.append({
            'Product URL': url,
            'Product Name': name.text,
            'Product Price': price.text,
            'Rating': rating,
            'Reviews': review
        })

for page in pageUrls:
    soup = getdata(page)
    content = soup.find_all('div', {'class': 'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16'})
    appendInfo(content)

data = data[0:200]

data = pd.DataFrame(data)

print(data)


                                           Product URL  \
0    https://www.amazon.in/Skybags-Brat-Black-Casua...   
1    https://www.amazon.in/American-Tourister-AMT-S...   
2    https://www.amazon.in/Backpack-Small-Black-Wat...   
3    https://www.amazon.in/Number-Backpack-Compartm...   
4    https://www.amazon.in/TRUE-HUMAN-Anti-Theft-ba...   
..                                                 ...   
195  https://www.amazon.in/HOEXL-Expandable-Laptop-...   
196  https://www.amazon.in/Classic-Leather-Laptop-B...   
197  https://www.amazon.in/Shalimar-Capacity-Travel...   
198  https://www.amazon.in/uppercase-Professional-r...   
199  https://www.amazon.in/Gear-Superior-Casual-Bac...   

                                          Product Name Product Price  \
0            Skybags Brat Black 46 Cms Casual Backpack          ₹659   
1    American Tourister Fizz Large Size 32 Ltrs Cas...        ₹1,199   
2    Mi Step Out 12 L Mini Backpack (Small Size, Bl...          ₹299   
3    FUR JADEN 

In [6]:
import requests

# use to parse html text
from lxml.html import fromstring
from itertools import cycle
import traceback


def to_get_proxies():
	# website to get free proxies
	url = 'https://free-proxy-list.net/'

	response = requests.get(url)

	parser = fromstring(response.text)
	# using a set to avoid duplicate IP entries.
	proxies = set()

	for i in parser.xpath('//tbody/tr')[:10]:

		# to check if the corresponding IP is of type HTTPS
		if i.xpath('.//td[7][contains(text(),"yes")]'):

			# Grabbing IP and corresponding PORT
			proxy = ":".join([i.xpath('.//td[1]/text()')[0],
							i.xpath('.//td[2]/text()')[0]])

			proxies.add(proxy)
		return proxies

proxies = to_get_proxies()

# to rotate through the list of IPs
proxyPool = cycle(proxies)

# insert the url of the website you want to scrape.
url = ''

for i in range(1, 11):

	# Get a proxy from the pool
	# proxy = next(proxyPool)
	print("Request #%d" % i)

	try:
		response = requests.get(url, proxies={"http": proxy, "https": proxy})
		print(response.json())

	except:

		# One has to try the entire process as most
		# free proxies will get connection errors
		# We will just skip retries.
  	 print("Skipping. Connection error")

links = data["Product URL"]

descriptions = []
asins = []
manufacturers = []
prodDescs = []

def clean(item):
  item = item.replace("  ", "")
  item = item.replace("\n", "")
  item = item.replace("\u200e", "")
  return item

def findASIN(soup):
  content = soup.find('div', {'class':'a-column a-span6 a-span-last'})
  if not content:
    asin = "-"
  else:
    head = soup.find_all('th', {'class':'a-color-secondary a-size-base prodDetSectionEntry'})
    value = soup.find_all('td', {'class':'a-size-base prodDetAttrValue'})
    for i in range(len(head)):
      if head[i].text == " ASIN ":
        asin = value[i].text
        asin = clean(asin)
  asins.append(asin)

def findManufacturer(soup):
  content = soup.find('div', {'class':'a-column a-span6 a-span-last'})
  manufacturer = "-"
  if not content:
    manufacturer = "-"
  else:
    head = soup.find_all('th', {'class':'a-color-secondary a-size-base prodDetSectionEntry'})
    value = soup.find_all('td', {'class':'a-size-base prodDetAttrValue'})
    for i in range(len(head)):
      if head[i].text == " Manufacturer ":
        manufacturer = value[i].text
        manufacturer = clean(manufacturer)
        break
  manufacturers.append(manufacturer)

def getAllInfo(soup):
  description = soup.find('ul', {'class':'a-unordered-list a-vertical a-spacing-mini'})
  if not description:
    description = "-"
  else:
    description = description.text
  descriptions.append(description)
  findASIN(soup)
  findManufacturer(soup)
  content = soup.find('table', {'id':'productDetails_techSpec_section_1'})
  if not content:
    desc = "-"
  else:
    prodDesc = content.find_all('td', {'class':'a-size-base prodDetAttrValue'})
    desc = ""
    for item in prodDesc:
      item = str(item.text)
      item = item.replace(" ", "")
      desc += item + " "
      desc = clean(desc)
  prodDescs.append(desc)

i = 1
for url in links:
  soup = getdata(url)
  sleep(randint(2,10))
  print("Product: ", i)
  i+= 1
  getAllInfo(soup)

data['Description'] = descriptions
data['ASIN'] = asins
data['Manufacturer'] = manufacturers
data['Product Description'] = prodDescs

data

data.to_csv("final_data.csv")

Request #1
Skipping. Connection error
Request #2
Skipping. Connection error
Request #3
Skipping. Connection error
Request #4
Skipping. Connection error
Request #5
Skipping. Connection error
Request #6
Skipping. Connection error
Request #7
Skipping. Connection error
Request #8
Skipping. Connection error
Request #9
Skipping. Connection error
Request #10
Skipping. Connection error
Product:  1
Product:  2
Product:  3
Product:  4
Product:  5
Product:  6
Product:  7
Product:  8
Product:  9
Product:  10
Product:  11
Product:  12
Product:  13
Product:  14
Product:  15
Product:  16
Product:  17
Product:  18
Product:  19
Product:  20
Product:  21
Product:  22
Product:  23
Product:  24
Product:  25
Product:  26
Product:  27
Product:  28
Product:  29
Product:  30
Product:  31
Product:  32
Product:  33
Product:  34
Product:  35
Product:  36
Product:  37
Product:  38
Product:  39
Product:  40
Product:  41
Product:  42
Product:  43
Product:  44
Product:  45
Product:  46
Product:  47
Product:  48
Prod